In [1]:
 
import time  
import random
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from lxml import etree
import pandas as pd
#driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

#reddit url
base_url="https://www.imdb.com/feature/genre/?ref_=nv_ch_gr"
#driver.get(base_url)
time.sleep(5)

#driver.get(str(base_url)+str(search_url))
data={'Moviename':[],'rating':[],'genre':[],'director':[],'cast':[],'writer':[],'summary':[]}

#header file
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
url=requests.get(base_url,headers=headers)
content=url.content
soup = BeautifulSoup(content,"html.parser")
dom = etree.HTML(str(soup))
title=list(zip(dom.xpath('//div[@class="article"]//div[@class="ab_ninja"]//div[@class="image"]/a/@href'),dom.xpath('//div[@class="article"]//div[@class="ab_ninja"]//div[@class="image"]/a/img/@title')))
import re

for t in title:
    
    url=requests.get(t[0],headers=headers)
    content=url.content
    soup = BeautifulSoup(content,"html.parser")
    page_links=[]
    page_links.append(t[0].replace('https://www.imdb.com',""))
    
    i=0
    while i < 1:
        
        met=soup.find(class_="lister-page-next next-page",href=True)['href']
        url=requests.get('https://www.imdb.com'+met,headers=headers)
        content=url.content
        soup = BeautifulSoup(content,"html.parser")
        page_links.append(met)
        i=i+1   
    print(str(t[1])+' genre')
    for li in page_links:
        
        url=requests.get('https://www.imdb.com'+li,headers=headers)
        content=url.content
        
        soup = BeautifulSoup(content,"html.parser")
    
        
        g_data = soup.find_all(class_= "lister-item mode-advanced")

     #   dom = etree.HTML(str(soup))
        for i in g_data:


                    try:
                        try:
                            
                        
                            title=i.find(class_="lister-item-header").find('a')
                            if title is not None:
                                title=title.text
                            else:
                                dirs='NA'
                       
                        except:
                            title='NA'
                        
                        try:
                            
                            ratings=i.find(class_= "inline-block ratings-imdb-rating").find('strong')                 
                            if ratings is not None:
                                ratings=ratings.text
                            else:
                                ratings='NA'
                       
                        except:
                            ratings='NA'
                            
                        try:
                            
                            genre=i.find(class_="genre")
                            if genre is not None:
                                genre=genre.text
                            else:
                                genre='NA'
                       
                        except:
                            genre='NA'
                        meta_link=i.find(class_="lister-item-header").find('a',href=True)
                        ml="https://www.imdb.com"+meta_link['href']
                        url=requests.get(ml,headers=headers)
                        content=url.content
                        soup = BeautifulSoup(content,"html.parser")
                        try:
                            
                            
                            dirs=soup.find(text=re.compile('Director')).parent.parent.find_all(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link") 
                            if dirs is not None:
                                dirs=[dirss.text for dirss in dirs]
                            else:
                                dirs='NA'
                        except:
                            dirs='NA'
                        try:
                            stars=soup.find(text=re.compile('Stars')).parent.parent.find_all(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
                            
                            if stars is not None:
                                stars=[star.text for star in stars]
                            else:
                                stars='NA'
                       
                        except:
                             stars='NA'
                        try:
                            
                            writer=soup.find(text=re.compile('Writer')).parent.parent.find_all(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
                            if writer is not None:
                                writer=[writers.text for writers in writer]
                            else:
                                writer='NA'
                       
                        except:
                             writer= 'NA'
                        try:
                            
                            summary=soup.find(class_='ipc-html-content ipc-html-content--base').find('div')
                            if summary is not None:
                                summary=summary.text
                            else:
                                summary='NA'
                                            
                        except:
                            
                            summary='NA'
                        data['Moviename'].append(title)
                        data['rating'].append(ratings)
                        data['genre'].append(genre)
                        data['director'].append(dirs)
                        data['cast'].append(stars)
                        data['writer'].append(writer)
                        data['summary'].append(summary) 
                    except Exception as e:
                            print(traceback.format_exc())
    #  print(data['Moviename'],data['rating'],data['genre'],data['director'],data['cast'],data['writer'],data['summary'])
    df = pd.DataFrame(data)
    df.to_csv(str(t[1])+'_genre_imdb_scrapped.csv')

In [ ]:
path = os.getcwd()
csv_files = glob.glob("../input/imdb-movie-descriptions/*.csv")
dfs = [pd.read_csv(f) for f in csv_files]
mergeddata = pd.concat(dfs,ignore_index=True)

In [ ]:
mergeddata.to_csv('merged_data.csv')

In [ ]:




# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import necessary libraries

import os
import glob
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import re
import nltk.corpus


from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

import pickle

nltk.download('stopwords')
stop = stopwords.words('english')
mergeddata=pd.read_csv('merged_data.csv')

df=mergeddata.drop('Unnamed: 0',axis=1)

df['all']=df['genre']+df['director']+df['cast']+df['writer']+df['summary']
df['all']=df['all'].apply(lambda x : str(x).replace('\n',''))
df['all']=df['all'].apply(lambda x : str(x).lower())
df['all']=df['all'].apply(lambda x : re.sub(r"([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x))

df['all']=df['all'].apply(lambda x : " ".join([word for word in x.split() if word not in (stop)]))
df.to_csv('df_pro.csv')

# Vectorizing pre-processed movie plots using TF-IDF
tfidfvec = TfidfVectorizer()
tfidfvec.fit(df['all'])
transformed=tfidfvec.transform(df['all'])
transformed=transformed.toarray()
pickle.dump(tfidfvec, open('tfidf_movieid.pickle','wb'))

df_t = pd.DataFrame(transformed,columns=tfidfvec.get_feature_names())
tsvd = TruncatedSVD(n_components=200)
tsvd.fit(df_t)
tsvd_mat=tsvd.transform(df_t)
pickle.dump(tsvd_mat, open('tsvd_mat.pickle','wb'))

pickle.dump(tsvd, open('tsvd.pickle','wb'))



### Prediction

In [ ]:
import nltk.corpus
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pickle
import re

nltk.download('stopwords')
stop = stopwords.words('english')
df=pd.read_csv('df_pro.csv')
tfidf_movieid=pickle.load(open('tfidf_movieid.pickle','rb'))
tsvd_mat=pickle.load(open('tsvd_mat.pickle','rb'))
tsvd=pickle.load(open('tsvd.pickle','rb'))
def preprocess(x):
    x=str(x).replace('\n','').lower()
    x=re.sub(r"([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x)
    x= " ".join([word for word in x.split() if word not in (stop)])
    return x

q = "drama scifi josh Friedman"
q=preprocess(q)
query_mat = tsvd.transform(tfidf_movieid.transform([q]))

dist = pairwise_distances(X=tsvd_mat , Y=query_mat, metric='cosine')
if np.min(dist) == 1.0:
    print('movie name unknown')
else :
    
    print(df['Moviename'][np.argmin(dist.flatten())])













In [ ]:
d